# 라이브러리 호출

In [70]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import random
import os
import seaborn as sns
import pdb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler


import matplotlib.font_manager as fm
from scipy.interpolate import CubicSpline

import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim

In [71]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# 데이터 불러오기 

In [72]:
data = pd.read_csv('./HPI_Datatable_FINAL.csv', sep = ",")
class_data = pd.read_csv('./class_description (1).csv 22-47-52-462.csv')
h_data = pd.read_csv('./Output_labeling_data_221129_FINAL.csv')

자이로 가속도를 제외한 컬럼들 삭제한다.

In [73]:
data = data.drop(['Mag_x', 'Mag_y', 'Mag_z', 'Roll', 'Pitch', 'Yaw', 'WD_Mac'], axis=1)

필요한 컬럼을 제외하고 삭제한다.

In [76]:
h_data.drop('Index', axis=1, inplace=True)
h_data.drop('filename', axis=1, inplace=True)
h_data.drop('Class', axis=1, inplace=True)

h_data = h_data.rename(columns={'Pname': '이름'})
h_data = h_data.rename(columns={'starttime_ms': '시작시간(ms)'})
h_data = h_data.rename(columns={'starttime': '시작시간'})
h_data = h_data.rename(columns={'endtime_ms': '종료시간(ms)'})
h_data = h_data.rename(columns={'endtime': '종료시간'})
h_data = h_data.rename(columns={'pose': '자세'})
h_data = h_data.rename(columns={'behavior': '행동'})
h_data = h_data.rename(columns={'sound': '소리'})

h_data['기타'] = np.nan

h_data.columns

h_data['자세'] = h_data['자세'].replace({
    1:'서다',
    2:'뒷다리만 앉다',
    3:'모든 발이 앉다 (머리는 들고있음)', 
    4:'두 앞발을 들어올리다',
    5:'모든 발과 머리가 바닥과 닿은 채로 눕다',
    6:'옆으로 등을 대고 눕다',
    0:'해당없음'
})

h_data['행동'] = h_data['행동'].replace({
    8:'해당없음', 
    9:'걷다', 
    17:'먹다',                                                   
    13:'킁킁대다'
})

h_data['소리'] = h_data['소리'].replace({
    20:'해당없음', 
    21:'짖다', 
    23:'으르렁거리다'
})

In [77]:
start_list = []
end_list = []
behavior_list = []

h_data['시작시간'] = pd.to_datetime(h_data['시작시간'])
h_data['종료시간'] = pd.to_datetime(h_data['종료시간'])

data['Htime'] = pd.to_datetime(data['Htime'])

시계열 타입으로 변경한다.

In [78]:
total_b = []
for i in range (len(h_data)) :
    # 자세
    posture = class_data[(class_data['구분1']=='자세') & (class_data['구분2']==h_data.iloc[i]['자세'])]['CLASS_NUM'].iloc[0]
    # 행동
    behavior = class_data[(class_data['구분1']=='행동') & (class_data['구분2']==h_data.iloc[i]['행동'])]['CLASS_NUM'].iloc[0]
    # 소리
    sound = class_data[(class_data['구분1']=='소리') & (class_data['구분2']==h_data.iloc[i]['소리'])]['CLASS_NUM'].iloc[0]
    # 종합 행동
    total_behavior = str(posture) + '_' + str(behavior) + '_' + str(sound)
    total_b.append(total_behavior)
    
h_data['total_behavior'] = total_b

각 행동에 대한 종합적인 정보를 나타내며, 코드에서는 이를 이용해 원본 센서 데이터에 행동 정보를 추가하고, 최종적으로 분석 및 처리를 위한 데이터셋을 구성

In [79]:
def assign_class(row):
    if row['total_behavior'] == '1_13_25':
        return 1
    elif row['total_behavior'] == '1_14_25':
        return 2
    elif row['total_behavior'] == '1_18_25':
        return 3
    elif row['total_behavior'] == '1_22_25':
        return 1
    elif row['total_behavior'] == '2_13_25':
        return 4
    elif row['total_behavior'] == '2_14_25':
        return 14
    elif row['total_behavior'] == '2_18_25':
        return 3
    elif row['total_behavior'] == '3_13_25':
        return 6
    elif row['total_behavior'] == '3_18_25':
        return 3
    elif row['total_behavior'] == '4_13_25': #두 앞발을 들어올리다 (0)
        return 0
    else:
        return 5  # 기본값

h_data['Class'] = h_data.apply(assign_class, axis=1)

h_data = h_data[h_data['Class'] != 14]

In [80]:
data.head()

,Htime,Cnt,Gyro_x,Gyro_y,Gyro_z,Acc_x,Acc_y,Acc_z,Pname
0,2022-10-25 19:35:53.840,146899,0.9618,-0.3664,-0.0763,-0.0352,0.0195,-0.9941,nana
1,2022-10-25 19:35:53.860,146900,0.9618,-0.0916,0.0153,-0.0400,0.0327,-0.9971,nana
2,2022-10-25 19:35:53.880,146901,0.5038,-0.3206,0.0305,-0.0386,0.0146,-0.9971,nana
3,2022-10-25 19:35:53.900,146902,0.9160,0.0000,0.0305,-0.0410,0.0273,-1.0024,nana
4,2022-10-25 19:35:53.920,146903,0.8550,0.1527,-0.0458,-0.0449,0.0176,-0.9917,nana


In [82]:
#1. 각 열의 이름을 가져와서 열 이름을 기반으로 새로운 빈 리스트를 동적으로 생성
for col in range(len(data.columns)):
    if col >= 2 and col <= 7:
        col_name = data.columns[col]
        globals()['{}_list'.format(col_name)] = []
    elif col == 8:  # Pname
        col_name = data.columns[col]
        globals()['{}_list'.format(col_name)] = []
        globals()['behavior_list'] = []

unique_names = data['Pname'].unique()

for name in unique_names:
    name_data = data[data['Pname'] == name]
    h_data = h_data[h_data['이름'] == name]


#     count = 0

    # 2. h_data의 일부 범위를 반복
    for i in range(len(h_data)):
        
#         count += 1
#         if count == 30:
#             break

        # 3. 해당 시간 범위와 '이름' 컬럼 값이 일치하는 label_data 추출
        label_data = data[(data['Htime'] >= h_data['시작시간'].iloc[i]) & (
                data['Htime'] <= h_data['종료시간'].iloc[i])]
        label_data = label_data[label_data['Pname'] == h_data['이름'].iloc[i]]

        # 4. label_data의 'behavior' 컬럼에 h_data의 'Class' 값을 할당
        label_data['behavior'] = h_data['Class'].iloc[i]

        if len(label_data) == 0:
            continue

        test_df_len = 0

            # 5. label_data가 비어있지 않은 경우, 다음 작업을 수행
        for cols in label_data.columns:  # ['Htime', 'Cnt', WD_Mac', 'Pname', 'behavior']
            if cols == 'Cnt' or cols == 'Htime' or cols == 'Pname' or cols == 'behavior':
                continue

            # 최소 시작시간부터 최대시간까지 시간 리스트 만들기
            datetime_list = pd.date_range(np.min(label_data['Htime']), np.max(label_data['Htime']), freq='20L')

            # 시간을 인덱스로 빈 데이터 프레임 만들어서 해당 시간에 값 넣기
            test_df = pd.DataFrame(index=datetime_list, columns=[cols])
            for tdf in range(len(test_df)):
                value = label_data[label_data['Htime'] == test_df.index[tdf]][cols]
                test_df.iloc[tdf, 0] = value.iloc[0] if not value.empty else np.nan

            # 결측치를 선형 보간법으로 처리 (Gyro_x, Gyro_y, Gyro_z, Acc_x, Acc_y, Acc_z) 및 50시퀀스 데이터 결합
            if cols in ['Gyro_x', 'Gyro_y', 'Gyro_z', 'Acc_x', 'Acc_y', 'Acc_z']:
                test_df[cols] = test_df[cols].astype(float)  # 데이터 타입을 float으로 변경
                test_df.interpolate(method='polynomial', order=2, limit_direction='both', inplace=True)
                
            # 이상치 처리
            for idx in range(0, len(test_df), 75):
                seq_df = test_df.iloc[idx: idx + 75]
                Q1 = seq_df[cols].quantile(0.25)
                Q3 = seq_df[cols].quantile(0.75)
                IQR = Q3 - Q1

                seq_df.loc[(seq_df[cols] < Q1 - 1.5 * IQR) | (seq_df[cols] > Q3 + 1.5 * IQR), cols] = np.nan
                seq_df.interpolate(method='linear', inplace=True)
                test_df.iloc[idx: idx + 75] = seq_df

            # 5.2. 시프트 연산을 통해 각 컬럼을 150
            for x in range(-1, -150, -1) : 
                test_df['Shift_'+ str(x)] = test_df[cols].shift(x)
        
            #5.3. 인덱스가 75번째인 행만 남김 (50% 중복)
            test_df.reset_index(drop=True, inplace=True)
            test_df = test_df.iloc[test_df.index%75 == 0]
        
            # 5.4. 결측값이 있는 행을 시퀀스의 평균값으로 채우기(바꿔도 가능)
            test_df.fillna(test_df.mean(), inplace=True)
    
            # 5.5. test_df의 길이를 계산하고, 해당 길이만큼 각 리스트에 값을 추가
            test_df_len = len(test_df)
            print("Processing: {} - Behavior: {} - test_df_len: {}".format(name, label_data['behavior'].iloc[0], test_df_len))
            test_df.apply(lambda x : globals()["{}_list".format(cols)].append(list(x)), axis = 1)
    
        for l in range(test_df_len) :
            globals()['behavior_list'].append(label_data['behavior'].iloc[0]) 
            globals()['Pname_list'].append(label_data['Pname'].iloc[0])
            
        
    print("After processing", name, "- data_x sample count:", len(globals()["Gyro_x_list"]))
    print("After processing", name, "- data_y sample count:", len(globals()['behavior_list']))

/tmp/ipykernel_364469/2682057481.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.loc[(seq_df[cols] < Q1 - 1.5 * IQR) | (seq_df[cols] > Q3 + 1.5 * IQR), cols] = np.nan
/tmp/ipykernel_364469/2682057481.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.interpolate(method='linear', inplace=True)


Processing: nana - Behavior: 1 - test_df_len: 104


/tmp/ipykernel_364469/2682057481.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.loc[(seq_df[cols] < Q1 - 1.5 * IQR) | (seq_df[cols] > Q3 + 1.5 * IQR), cols] = np.nan
/tmp/ipykernel_364469/2682057481.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.interpolate(method='linear', inplace=True)


Processing: nana - Behavior: 1 - test_df_len: 104


/tmp/ipykernel_364469/2682057481.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.loc[(seq_df[cols] < Q1 - 1.5 * IQR) | (seq_df[cols] > Q3 + 1.5 * IQR), cols] = np.nan
/tmp/ipykernel_364469/2682057481.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.interpolate(method='linear', inplace=True)


...

Processing: nana - Behavior: 2 - test_df_len: 3
Processing: nana - Behavior: 2 - test_df_len: 3
Processing: nana - Behavior: 2 - test_df_len: 3


/tmp/ipykernel_364469/2682057481.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.loc[(seq_df[cols] < Q1 - 1.5 * IQR) | (seq_df[cols] > Q3 + 1.5 * IQR), cols] = np.nan
/tmp/ipykernel_364469/2682057481.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.interpolate(method='linear', inplace=True)
/tmp/ipykernel_364469/2682057481.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.loc[(seq_df[cols] < Q1 - 1.5 * 

Processing: nana - Behavior: 2 - test_df_len: 3
Processing: nana - Behavior: 2 - test_df_len: 3
After processing nana - data_x sample count: 10864
After processing nana - data_y sample count: 10864


/tmp/ipykernel_364469/2682057481.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.loc[(seq_df[cols] < Q1 - 1.5 * IQR) | (seq_df[cols] > Q3 + 1.5 * IQR), cols] = np.nan
/tmp/ipykernel_364469/2682057481.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq_df.interpolate(method='linear', inplace=True)


After processing janggun - data_x sample count: 10864
After processing janggun - data_y sample count: 10864
After processing bomsoon - data_x sample count: 10864
After processing bomsoon - data_y sample count: 10864
After processing purong - data_x sample count: 10864
After processing purong - data_y sample count: 10864
After processing potato - data_x sample count: 10864
After processing potato - data_y sample count: 10864
After processing bordi - data_x sample count: 10864
After processing bordi - data_y sample count: 10864
After processing ddolbok - data_x sample count: 10864
After processing ddolbok - data_y sample count: 10864
After processing choko - data_x sample count: 10864
After processing choko - data_y sample count: 10864
After processing momo - data_x sample count: 10864
After processing momo - data_y sample count: 10864


In [ ]:
test_df.head()

In [83]:
label_data.head()

,Htime,Cnt,Gyro_x,Gyro_y,Gyro_z,Acc_x,Acc_y,Acc_z,Pname,behavior
2045638,2022-11-27 19:23:43.431,749265,50.030499,-64.610703,-117.038200,-0.0972,0.6895,-0.6616,nana,2
2045639,2022-11-27 19:23:43.411,749264,-166.106903,243.679398,-65.374001,0.1919,0.3042,-0.0146,nana,2
2045640,2022-11-27 19:23:43.391,749263,-180.824402,312.274811,-107.190804,-0.0688,0.4004,0.7690,nana,2
2045641,2022-11-27 19:23:43.371,749262,-201.725204,197.603104,-15.450400,-1.6455,1.1514,0.9082,nana,2
2045642,2022-11-27 19:23:43.351,749261,-108.961800,-433.450409,273.450409,-2.9248,0.9839,2.4717,nana,2


In [37]:
total_data = []

# 각 컬럼에 대하여
for col in range(len(label_data.columns)) : 
    col_name = label_data.columns[col]
    
    # 2번째부터 7번째 컬럼까지 해당하는 경우
    if col >= 2 and col <= 7 :
        # 전역 변수로 선언한 각 컬럼 이름에 대응하는 리스트를 total_data에 추가
        total_data.append(globals()['{}_list'.format(col_name)])

In [68]:
len(total_data[0])

10864

1. 빈 데이터프레임 생성: create_empty_df 함수는 주어진 인덱스 목록과 열 이름을 사용하여 빈 데이터프레임을 생성합니다.

2. 센서 데이터 처리: process_sensor_data 함수는 각 센서 데이터 열에 대해 시간 인덱스를 생성하고, NaN 값을 처리합니다. 이 작업은 선형 보간법을 사용하여 수행됩니다.

3. 시프트 및 필터링: shift_and_filter 함수는 주어진 데이터프레임을 시프트 연산을 수행한 후, NaN 값을 포함하는 행을 제거합니다. 이 작업은 시계열 데이터를 처리하는 데 사용됩니다.

4. 데이터 저장: 이 작업은 전체 데이터셋에 대해 실행되며, 각 센서 데이터와 메타데이터를 저장하기 위한 리스트를 생성합니다.

5. 각 행동 데이터 처리: 주어진 행동 데이터 범위에 대해, 센서 데이터를 처리하고 시프트 연산 및 필터링을 수행한 후, 각 리스트에 행동과 이름 정보를 추가합니다.

In [38]:
for i, data in enumerate(total_data):
    print(f'Shape of data {i}: {np.array(data).shape}')

Shape of data 0: (10864, 150)
Shape of data 1: (10864, 150)
Shape of data 2: (10864, 150)
Shape of data 3: (10864, 150)
Shape of data 4: (10864, 150)
Shape of data 5: (10864, 150)


# X_tain Data만들기

In [45]:
print(np.array(total_data).shape)

(6, 10864, 150)


In [39]:
# total_data를 변환하여 total_x_train을 만듦
total_x_train = np.transpose(total_data,(1,2,0))
print(np.array(total_x_train).shape)

(10864, 150, 6)


In [40]:
# 각 값의 출현 빈도를 계산합니다.
count_dict = {}
for value in behavior_list:
    if value in count_dict:
        count_dict[value] += 1
    else:
        count_dict[value] = 1

# 값별로 정렬된 결과를 출력합니다.
for value, count in sorted(count_dict.items()):
    print(f"{value}: {count}")

0: 379
1: 2125
2: 159
3: 589
4: 3751
5: 1363
6: 2498


# Y_tain Data 만들기

In [41]:
# behavior_list를 정수 값과 원-핫 벡터로 변환
values = np.array(behavior_list)

# LabelEncoder를 사용하여 행동 레이블을 정수로 변환합니다. 이렇게 하면 각 레이블에 고유한 정수가 할당
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

# 변환된 정수 레이블을 2D 배열로 바꾸어 OneHotEncoder에 입력으로 제공
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

# OneHotEncoder를 사용하여 정수 레이블을 원-핫 벡터로 변환
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# 원-핫 인코딩된 행동 레이블을 y_train (total_y_train)으로 사용할 수 있는 형태로 변환
total_y_train = onehot_encoded
print("로드된 y 데이터 길이 : ", np.array(total_y_train).shape)

로드된 y 데이터 길이 :  (10864, 7)


/home/user/anaconda3/envs/Junhyeok/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [42]:
# 변환된 데이터를 저장
np.save('dmlab_x_data.npy', total_x_train)
np.save('dmlab_y_data.npy', behavior_list)